In [1]:
import os
os.environ["OMP_NUM_THREADS"]    = "4"
os.environ["MKL_NUM_THREADS"]    = "4"
os.environ["OPENBLAS_NUM_THREADS"] = "4"

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
import pandas as pd
import numpy as np
from ExKMC.Tree import Tree
from IPython.display import Image
import time

In [2]:
import os

print("Number of threads used by OMP:", os.environ.get("OMP_NUM_THREADS"))
print("Number of threads used by MKL:", os.environ.get("MKL_NUM_THREADS"))
print("Number of threads used by OPENBLAS:", os.environ.get("OPENBLAS_NUM_THREADS"))

Number of threads used by OMP: 4
Number of threads used by MKL: 4
Number of threads used by OPENBLAS: 4


In [17]:

df = pd.read_csv("/Users/bojitha/Desktop/FYP/exKMC/ExKMC/notebooks/datasets/sub_data_2000000.csv")
# df = pd.read_csv("experiment_results/subset_features_28_datapoints_3000000.csv")


In [18]:
# df = df.drop(df.columns[0], axis=1)
# df.columns = [f'feature {i+1}' for i in range(df.shape[1])]
# print(df.columns)

df.columns

Index(['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10',
       'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20',
       'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30',
       'f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40',
       'f41', 'f42', 'f43', 'f44', 'f45', 'f46', 'f47', 'f48', 'f49', 'f50',
       'f51', 'f52', 'f53', 'f54', 'f55', 'f56', 'f57', 'f58', 'f59', 'f60',
       'f61', 'f62', 'f63', 'f64', 'f65', 'f66', 'f67', 'prediction'],
      dtype='object')

In [19]:
df.shape

(2000000, 69)

In [20]:
# # 1 
# cluster_centers = np.array([[5.305993690851735, 3.6829652996845423, 1.197160883280757, 0.20425867507886436, 3.4562302839116716, 0.5193217665615142, 0.0, 1.748422712933754, 1.7795741324921135, 1.71096214511041, 0.002365930599369085, 2.5094637223974763, 0.7113564668769716, 0.26774447949526814, 0.0, 6.200315457413249, 0.11356466876971609, 0.007097791798107256, 0.0011829652996845426, 0.15812302839116718, 0.32689274447949523, 0.12381703470031545, 0.11080441640378548, 0.032334384858044164, 1.585173501577287, 0.0031545741324921135, 1.2543375394321765, 1.724763406940063, 1.0488958990536277, 0.0011829652996845426, 0.0, 3.852129337539432, 1.4858044164037854, 1.8450315457413249, 0.9676656151419558, 0.001971608832807571, 1.8414826498422712, 3.2381703470031544, 1.6569400630914826, 1.221608832807571, 2.4077287066246056, 0.22594637223974762, 2.5402208201892744, 0.3659305993690852, 0.0, 0.0, 5.7208201892744475, 0.0, 1.2421135646687698, 51.02917981072555, 0.014195583596214511, 2.447160883280757, 0.34384858044164035, 1.1537854889589905, 0.001971608832807571, 0.7133280757097792, 0.04574132492113565, 0.04140378548895899, 2.5508675078864353, 0.0, 0.001971608832807571, 0.12263406940063092, 1.8568611987381702, 1.8789432176656151, 0.030757097791798107, 7.7799684542586744, 5.21096214511041, 0.044558359621451105], [1.2130973451327434, 8.35646017699115, 1.3897345132743364, 0.05557522123893805, 0.264070796460177, 0.06902654867256637, 0.09911504424778761, 0.21415929203539824, 0.21628318584070796, 0.1550442477876106, 0.0, 0.07787610619469026, 0.17734513274336283, 0.06761061946902655, 0.029026548672566373, 0.24035398230088495, 0.03964601769911504, 0.001415929203539823, 0.0, 0.005663716814159292, 0.008849557522123894, 0.004601769911504425, 0.0017699115044247787, 0.008141592920353982, 0.3345132743362832, 0.0, 1.3313274336283185, 0.14123893805309734, 4.0, 0.0, 0.04353982300884956, 0.4361061946902655, 1.1635398230088496, 0.3023008849557522, 0.1582300884955752, 0.0, 0.295929203539823, 0.2895575221238938, 1.497345132743363, 1.1773451327433628, 1.9826548672566373, 0.08070796460176992, 2.7812389380530975, 0.1111504424778761, 221.2046017699115, 0.02831858407079646, 0.49805309734513276, 0.8102654867256637, 0.12106194690265487, 16.972389380530974, 1.0, 0.919646017699115, 0.0, 1.5589380530973451, 0.0, 0.49911504424778763, 0.5791150442477876, 0.20353982300884957, 0.2208849557522124, 0.05911504424778761, 0.0, 0.0431858407079646, 0.17805309734513275, 0.44672566371681416, 0.0, 3.427610619469027, 0.4938053097345133, 0.0], [1.205970463026226, 3.234088400693751, 1.765911599306249, 0.029642087559783467, 0.13890786776685762, 0.07431544647080465, 0.13974877805224156, 0.19214800021022757, 0.19351447942397645, 0.14411100015767067, 0.0, 0.06974299679402954, 0.1275555789141746, 0.06995322436537552, 0.03868187312766069, 0.11268197824144636, 0.047669101802701425, 0.0005781258212014506, 0.00021022757134598203, 0.015661954065275662, 0.0033636411415357125, 0.0014190361065853788, 5.255689283649551e-05, 0.010143480317443634, 0.3615914227150891, 0.0, 1.3323697902979976, 0.10947600777842015, 4.0, 0.0, 0.06753560729489673, 0.3887633363115573, 1.0210753140274347, 0.29568507909812375, 0.1816366216429285, 0.0, 0.2943711567772113, 0.14195616755137436, 1.9063436169653651, 1.0844063698954118, 1.9376149682030799, 0.09612655699795028, 2.0915541073211754, 0.008724444210858254, 116.01303410942346, 0.03763073527093078, 0.40095653544962423, 0.9921164660745257, 0.1332317233405161, 13.520838808009671, 1.0, 0.8908393335785989, 0.00015767067850948652, 1.6214327008987228, 0.00015767067850948652, 0.4853103484521995, 0.02711935670363168, 0.009775582067588164, 0.1708099017186104, 0.0713722604719609, 0.0, 0.05429127030009986, 0.146055605192621, 0.37378462185315603, 0.0, 3.388395438061702, 0.37078887896147583, 0.00015767067850948652], [4.305243849805783, 3.0263271471730686, 1.5752769385699898, 0.14116673859876278, 0.0, 2.0971083297367286, 2.758775715724356, 1.95281254495756, 1.9971586822039995, 0.07808229031793987, 0.02938426125737304, 1.2012660048913826, 0.04920155373327579, 2.953316069630269, 2.796288303841174, 0.0, 1.6767011940727954, 0.08880017263703065, 0.023413897280966767, 0.26435045317220546, 0.04290749532441376, 0.015321536469572723, 0.03262120558193066, 0.053121852970795566, 6.562796719896418, 0.02949215940152496, 1.9396849374190763, 0.12167314055531578, 1.2682707524097252, 0.013631132211192633, 1.6285786217810387, 3.6666666666666665, 1.43033376492591, 1.99269889224572, 3.0558552726226442, 0.0009351172493166451, 1.9770896273917422, 0.0, 1.9247590274780606, 1.1152711840023017, 1.4387498201697597, 2.76348726801899, 1.216515609264854, 0.051611278952668684, 0.0, 0.9848223277226298, 1.2624442526255215, 0.0, 2.8446986045173355, 10.0, 0.00046755862465832254, 2.667637749964034, 0.8656668105308589, 1.1278233347719753, 0.017839159833117537, 0.45734426701194075, 0.026506977413321826, 0.016328585814990648, 0.1827075240972522, 2.9434254064163428, 0.05725794849661919, 1.5817508272191052, 1.0266508416055244, 1.0475111494748957, 0.02416918429003021, 10.76686807653575, 1.0613221119263416, 0.17119838872104734], [4.413358369098712, 10.899678111587983, 1.2204935622317596, 0.22934549356223177, 0.00509656652360515, 2.191255364806867, 2.271727467811159, 1.8857296137339055, 1.9423283261802575, 0.06947424892703863, 0.032457081545064374, 1.4704935622317596, 0.024946351931330473, 2.6531652360515023, 2.3189377682403434, 0.024946351931330473, 1.4884656652360515, 0.07054721030042918, 0.006437768240343348, 0.16228540772532188, 0.0651824034334764, 0.03889484978540773, 0.05793991416309013, 0.05874463519313305, 6.921942060085837, 0.03594420600858369, 1.9468884120171674, 0.39028969957081544, 1.3342274678111588, 0.03460300429184549, 1.4369635193133048, 3.4788090128755367, 1.5614270386266094, 1.9586909871244635, 3.0085836909871246, 0.002145922746781116, 1.9364270386266094, 0.00509656652360515, 1.8039163090128756, 1.2615343347639485, 1.5705472103004292, 2.4401824034334765, 1.7414163090128756, 0.050160944206008584, 0.0, 0.7913090128755365, 2.0834227467811157, 0.0, 2.6231223175965663, 22.665236051502145, 0.007242489270386266, 2.729881974248927, 1.013412017167382, 1.1410944206008584, 0.06410944206008584, 0.49946351931330474, 0.028165236051502146, 0.017435622317596565, 0.6070278969957081, 2.5190450643776825, 0.07027896995708155, 1.3776824034334765, 1.099785407725322, 1.2022532188841202, 0.029506437768240343, 10.382778969957082, 1.3004291845493563, 0.2709227467811159], [1.1815935762816554, 2.0951204447189626, 1.7572575663990118, 0.0420012353304509, 0.0, 0.06979617047560223, 0.11303273625694873, 0.18838789376158124, 0.19024088943792467, 0.15626930203829525, 0.0, 0.06825200741198272, 0.31377393452748614, 0.07659048795552811, 0.029956763434218655, 0.0, 0.04972205064854849, 0.0009264978381717109, 0.0, 0.008647313156269303, 0.005250154416306363, 0.005250154416306363, 0.0006176652254478073, 0.01266213712168005, 0.3709079678814083, 0.0, 1.2615812229771464, 0.1102532427424336, 4.0, 0.0, 0.09203211859172329, 0.3817171093267449, 1.0463248919085857, 0.28196417541692403, 0.18807906114885733, 0.0, 0.27733168622606547, 0.0, 1.634959851760346, 1.11550339715874, 1.8872760963557753, 0.09882643607164918, 2.8032736256948736, 0.11673872760963558, 219.1729462631254, 0.025015441630636195, 0.40920321185917236, 0.8069796170475603, 0.14113650401482397, 10.0, 1.0, 0.8486720197652873, 0.0, 1.525633106856084, 0.0, 0.47344039530574433, 0.583384805435454, 0.2038295243977764, 0.1778875849289685, 0.06300185299567634, 0.0, 0.054354539839407044, 0.14144533662754788, 0.3875849289684991, 0.0, 3.241507103150093, 0.355775169857937, 0.0], [5.917483980646004, 2.459134301033085, 1.5829737151824244, 0.10108539296456126, 0.021707859291225314, 0.562181247548058, 0.0011769321302471556, 1.6484896037661827, 1.6981822937099516, 0.12436249509611612, 0.019484765267425132, 2.2552635020269385, 0.08748528834837191, 0.26585589119916303, 0.0005230809467765136, 0.1326010200078462, 0.12684712959330455, 0.009546227278671373, 0.0028769452072708247, 0.3057408133908722, 0.5472734405649273, 0.07976984438341833, 0.2417941676474434, 0.06250817313979337, 1.7507519288609912, 0.03844644958807375, 1.917745521119393, 1.7763828952530403, 0.8239832614097031, 0.004969268994376879, 0.00039231071008238524, 3.5545965738197984, 1.3720413233947952, 1.82895253040408, 0.8480449849614227, 0.005100039231071008, 1.8653066562050475, 0.021707859291225314, 1.75389041454165, 1.1239701843860337, 2.3925722505557734, 0.25670197463057404, 1.6754282725251732, 0.10605466195893813, 0.0, 0.0010461618935530272, 5.851052700405387, 0.0, 1.8169216686282201, 22.489342225709425, 0.007976984438341833, 2.1686936053354255, 1.7340133385641427, 1.0893160716620895, 0.0036615666274355954, 0.659997384595266, 0.014907806983130639, 0.011115470119000914, 2.584150647312671, 0.0010461618935530272, 0.01870014384726036, 0.13482411403164637, 1.8537988753759642, 1.8999607689289917, 0.14855498888452986, 9.277363672028246, 4.920753236563358, 0.2343402641558781], [4.478732030962035, 1.0811647622558054, 1.7681533357906376, 0.12635458901584962, 0.013711758201253225, 2.268337633615923, 2.742720235901216, 1.9486914854404718, 1.9957242904533725, 0.08573534832288979, 0.051013638039071135, 1.1690379653520089, 0.08293402137854773, 3.093918171765573, 2.9007003317360853, 0.08787320309620346, 1.840987836343531, 0.10563951345374124, 0.014375230372281606, 0.33063029856247694, 0.0450423884998157, 0.011131588647253962, 0.05042388499815702, 0.05410984150387025, 7.021083671212679, 0.03781791374861777, 1.9371175820125321, 0.1432362698120162, 1.0976778474014006, 0.03251013638039071, 1.7687430888315516, 3.4830814596387762, 1.5708809436048654, 1.9919646148175452, 2.561813490600811, 0.0010320678215997052, 1.9831183192038333, 0.013711758201253225, 1.900331736085514, 1.1731662366384077, 1.4590490232215259, 2.939550313306303, 1.4005897530409142, 0.03346848507187615, 0.0, 0.9213416881680796, 1.4066347217102837, 0.0, 3.043199410246959, 22.38776262440103, 0.0005897530409141172, 2.45912274235164, 1.2069295982307409, 1.1453004054552156, 0.055584224106155544, 0.4502027276078142, 0.013195724290453372, 0.008182823442683377, 0.21422779211205306, 2.9497235532620714, 0.08853667526723184, 1.5832657574640618, 1.0238112790269074, 1.0586804275709547, 0.03427939550313306, 11.669738297088093, 1.0602285293033542, 0.29332841872465903], [4.315645399031375, 4.105917035165298, 1.1918298589176668, 0.12802695304274583, 3.4613602863760793, 2.0305327437355234, 2.8323857654243, 1.9747315224257738, 1.9970520109496737, 1.445146346599284, 0.008843967150979154, 1.1316066540324279, 0.8450200042114129, 2.903137502632133, 2.786902505790693, 5.254579911560328, 1.618867129922089, 0.08907138344914718, 0.006738260686460307, 0.2078332280480101, 0.034533586018109075, 0.011581385554853654, 0.022952200463255423, 0.030111602442619498, 6.54979995788587, 0.005895978100652769, 1.2413139608338597, 0.1339229311433986, 1.1962518424931565, 0.011370814908401769, 1.852389976837229, 3.874078753421773, 1.59107180459044, 1.9909454622025688, 3.1284480943356496, 0.0002105706464518846, 1.954306169719941, 3.405137923773426, 1.8848178563908191, 1.2769004000842281, 1.2945883343861866, 2.6864603074331437, 1.7706885660138976, 0.22362602653190145, 0.0, 0.9856811960412718, 1.3217519477784796, 0.0, 2.7620551695093702, 50.69488313329122, 0.0012634238787113076, 2.6085491682459465, 0.2568961886712992, 1.1781427668982944, 0.017687934301958308, 0.41461360286376076, 0.03979785217940619, 0.03074331438197515, 0.19709412507896398, 3.3099599915771742, 0.020425352705832806, 1.5011581385554853, 1.0444304064013477, 1.030953885028427, 0.007369972625815961, 10.069698883975574, 1.0768582859549378, 0.07033059591492946], [5.3197365768429545, 3.622337208520933, 1.4312882198776964, 0.20583294133458774, 0.0, 0.672266648746724, 0.0007391976345675694, 1.6105100463678517, 1.6553995027215913, 0.11827162153081111, 0.009071970969692898, 2.161884281970298, 0.054028627108393255, 0.40844029299106244, 0.0002687991398427525, 0.0, 0.18137221960889727, 0.0118943619380418, 0.0071903769907936295, 0.22404408305893422, 0.4441233788051878, 0.10483166453867349, 0.1829850144479538, 0.05974060883005175, 2.1216316107788455, 0.01975673677844231, 1.8910691485787245, 1.7003561588602918, 1.2730999260802367, 0.0030239903232309657, 0.0002687991398427525, 3.5580270143135544, 1.2958806531819098, 1.7860358846851692, 1.176265035951885, 0.002755191183388213, 1.8120422014649553, 0.0, 1.6490827229352867, 1.0637725959276931, 2.30172703447349, 0.37504199986560044, 2.004502385592366, 0.13171157852294874, 0.0, 0.0002015993548820644, 5.580337342920503, 0.0, 1.6184396209932128, 10.0, 0.026006316779786307, 2.540689469793697, 1.163564276594315, 1.1252603991667227, 0.0030239903232309657, 0.6556011020764734, 0.03346549291042269, 0.020899133122774007, 2.5355150863517237, 0.000537598279685505, 0.009609569249378402, 0.20159935488206437, 1.7325448558564613, 1.8532356696458572, 0.09609569249378402, 8.406961897721928, 4.528055910221087, 0.148242725623278]])
# 2
# cluster_centers = np.array([[4.299245363761493, 3.1214827021278637, 1.568816246235601, 0.18686747719005783, 0.0, 2.079331014814886, 2.485138807719453, 1.939380003896133, 1.98556834040705, 0.08195611421417873, 0.02690567206696239, 1.2632029406222955, 0.05341215018634374, 2.799147960438283, 2.5011289204837595, 0.0, 1.5686597622081493, 0.085223117481182, 0.024162411259185453, 0.25363186653509234, 0.05787354174450949, 0.022514538643570902, 0.040989234537621634, 0.053533505146408374, 6.563182821247338, 0.027710447065285776, 1.93759161501097, 0.2817798301669269, 1.3294372004049424, 0.012435689855044695, 1.470633696440148, 3.6802296802296803, 1.433435207628756, 1.9884010206590852, 3.0785102720586592, 0.0009740332320977483, 1.9725354564064241, 0.0, 1.8949289594450884, 1.1068785907495586, 1.4984048209854661, 2.6182077149819087, 1.347467992629283, 0.05269040752911721, 0.0, 0.8812445909220104, 1.6971938262260844, 0.0, 2.739463674947546, 10.0, 0.0010379042637107155, 2.748571684055555, 0.8559835979190819, 1.144527370333822, 0.016612855322532743, 0.4738080867113125, 0.030466482079385307, 0.018449147481405546, 0.4288588482136869, 2.6356189581996032, 0.05247643957321377, 1.4939466229788811, 1.057107089365154, 1.1351798448572643, 0.02771683416844707, 10.639556607298543, 1.1812564070628586, 0.16507468120371346], [1.1760542089489459, 3.1747002997002998, 1.7731479047268521, 0.02645380934854619, 0.0, 0.07059387980440612, 0.12829933224670068, 0.1779339081970661, 0.17928781744571218, 0.10255533939744466, 0.0, 0.06722882380777118, 0.11369551501130448, 0.06455386718544613, 0.035227929964772066, 0.0, 0.04462642620537357, 0.0010647247489352753, 0.00019717124980282873, 0.015287344234712655, 0.0028984173721015825, 0.0013867711236132288, 0.0002891844997108155, 0.009181607865818392, 0.3398575109101425, 0.0, 1.2977022977022976, 0.10308770177191229, 4.0, 0.0, 0.06549371680950627, 0.3595680635154319, 0.9402702560597297, 0.27450181397549817, 0.17013907145486093, 0.0, 0.2726746937273253, 0.0, 1.911226930963773, 1.072236973552763, 1.9406711709343287, 0.09075792628424208, 2.089410589410589, 0.006703822493296177, 115.85474394027025, 0.03265155896734844, 0.3720818655029181, 0.9910747147589253, 0.12608444187391554, 10.0, 1.0, 0.8263447079236552, 6.572374993427625e-05, 1.5801632577948368, 6.572374993427625e-05, 0.4862308743887691, 0.030259214469740784, 0.010890425364109575, 0.16078000946422, 0.06872075293127924, 0.0, 0.051238235448761764, 0.13419475261580524, 0.3578263841421736, 0.0, 3.2395433513854566, 0.3392002734107997, 6.572374993427625e-05], [1.21117941712204, 1.253358378870674, 1.8584357923497268, 0.04479735883424408, 0.0, 0.07724271402550091, 0.1366974043715847, 0.20605646630236793, 0.20804872495446267, 0.15163934426229508, 0.0, 0.07994649362477231, 0.299408014571949, 0.07778346994535519, 0.03864981785063752, 0.0, 0.0521117941712204, 0.001223816029143898, 2.8460837887067395e-05, 0.009847449908925318, 0.008339025500910747, 0.004639116575591985, 0.001138433515482696, 0.009676684881602914, 0.4078153460837887, 0.0, 1.318362932604736, 0.11617714025500911, 4.0, 0.0, 0.08176798724954462, 0.4179474043715847, 1.1109972677595628, 0.31429303278688525, 0.19376138433515483, 0.0, 0.31087773224043713, 0.0, 1.6339367030965393, 1.1061304644808743, 1.9308686247723132, 0.10550432604735883, 2.7465277777777777, 0.09420537340619307, 219.82553506375228, 0.033043032786885244, 0.43772768670309653, 0.8196436703096539, 0.1470856102003643, 11.997580828779599, 1.0, 0.9474043715846995, 0.00017076502732240437, 1.5707536429872495, 0.00017076502732240437, 0.4847165300546448, 0.546875, 0.19205373406193077, 0.1866177140255009, 0.07729963570127504, 0.0, 0.05823087431693989, 0.15636384335154827, 0.42161885245901637, 0.0, 3.418545081967213, 0.39426798724954465, 0.00017076502732240437], [4.302700092382632, 4.115919207189048, 1.1999454102628706, 0.16198454690518183, 3.4622490971697317, 2.009994121105232, 2.735386747291509, 1.9706055261610815, 1.9958427815570672, 1.4456622155034853, 0.008041488200218359, 1.1321911480641638, 0.8166624674561181, 2.8806794322667337, 2.715041572184429, 5.194150499706055, 1.5715125556395397, 0.0867556899302931, 0.005878894767783656, 0.20214999580078943, 0.03159905937683715, 0.013563450071386578, 0.020618123792726966, 0.03149407911312673, 6.5290165448895605, 0.005605946082136558, 1.2484672881498278, 0.16819937851683883, 1.2429243302259174, 0.009931132947005962, 1.8840388007054671, 3.8782648862013938, 1.6045183505500964, 1.9885151591500796, 3.1373561770387166, 0.0004829092130679432, 1.9526539010665993, 3.408310237675317, 1.8637566137566137, 1.2715209540606365, 1.322562358276644, 2.624758545393466, 1.894662803392962, 0.21541950113378683, 0.0, 0.9549004787100025, 1.4076593600403122, 0.0, 2.6995044931552865, 50.71248005374989, 0.0013017552700092382, 2.667338540354413, 0.24680859998320315, 1.189594356261023, 0.017426723775930124, 0.42002603510540015, 0.03689006466784244, 0.028659611992945325, 0.24979003947257913, 3.180629041740153, 0.018371546149323927, 1.4566221550348533, 1.056458385823465, 1.0526161081716636, 0.006676744771982867, 10.040375409423028, 1.0975896531452087, 0.06800621483161165], [5.808677831127767, 3.3160089003396185, 1.433159620564469, 0.062407776086192766, 3.513291954561424e-05, 0.21143576531209746, 2.9277432954678535e-05, 1.5696568684857712, 1.6080454385759457, 0.10923410235390561, 0.00990162782527228, 2.324669165007612, 0.05217824101182809, 0.04936175196158801, 1.1710973181871414e-05, 0.0002576414100011711, 0.0192352734512238, 0.0016161142990982552, 0.003378615762969903, 0.2567220986063942, 0.5529335987820588, 0.10232462817660148, 0.2143635086075653, 0.0545907014872936, 0.5705644689073662, 0.021858531443962995, 1.90018737557091, 1.7308701253074132, 1.0285689190771754, 0.0019264550884178476, 1.1710973181871414e-05, 3.5416852090408715, 1.2715833235741891, 1.75757114416208, 0.42871530624194876, 0.004625834406839209, 1.796275910528165, 3.513291954561424e-05, 1.6870886520669868, 1.0630694460709686, 2.473334114064879, 0.04535074364679705, 1.8119686145918728, 0.13500995432720458, 0.0, 2.3421946363742828e-05, 5.726765429207167, 0.0, 1.6117402506148262, 13.87909591287036, 0.024557910762384356, 2.2828609907483313, 1.3444080103056564, 1.0794940859585431, 0.0016161142990982552, 0.6744525120037476, 0.021395948003279073, 0.013772104461880784, 2.529874692586954, 1.1710973181871414e-05, 0.00957372057617988, 0.024821407658976464, 1.896726782995667, 1.85237732755592, 0.11640121794121092, 8.379845415154, 5.379997657805364, 0.16883710036304017], [1.3215700974617115, 3.3223733533254793, 1.7589696904787404, 0.03636071543322266, 0.7176020134946984, 0.10013923101638642, 0.18630181000321303, 0.2691174895576738, 0.27091142765342185, 0.30033736746278245, 0.0, 0.10204026989397022, 0.23682660383420798, 0.09515904466102602, 0.05181000321302345, 0.5968726571703973, 0.0660276320017136, 0.001044232622898147, 0.00024097675913034165, 0.02158080753989504, 0.004016279318839027, 0.0022223412230909286, 0.0004284031273428296, 0.012289814715647423, 0.5154225125843418, 0.0, 1.4489664774552853, 0.15738459890757203, 4.0, 0.0, 0.09333833136981899, 0.5435632430116739, 1.3471136339295275, 0.4097943664988754, 0.23690692942058475, 0.0, 0.4069026453893113, 0.748125736317875, 1.8879725821998499, 1.1131252008139658, 1.9422994537860125, 0.13309949662632536, 2.1245849844703866, 0.013842776052265181, 116.42719824354717, 0.049346685230802184, 0.5752382992395845, 0.9907893327621291, 0.18367784084823818, 28.23878119310271, 1.0, 1.2339616579201027, 0.0001606511727535611, 1.7988647317125415, 0.0001606511727535611, 0.48679982863874904, 0.032076684159794366, 0.011861411588304595, 0.24389525543536467, 0.10469101424440398, 0.0, 0.0734979115347542, 0.20539252436542785, 0.47073471136339295, 0.0, 3.9936275034807753, 0.5255435364678162, 0.0001606511727535611], [1.183960589383987, 10.892153381856915, 1.208237173797266, 0.04500266287946032, 0.0, 0.059914787857269655, 0.07584768329486952, 0.18617965560092312, 0.18795490857447184, 0.05290253861175217, 0.0, 0.07757855494407953, 0.03763536303923309, 0.05241434404402627, 0.022279424818036568, 0.0, 0.033507899875732286, 0.0005769572164033374, 4.4381324338718264e-05, 0.002840404757677969, 0.006834723948162613, 0.0067015799751464575, 0.0010651517841292384, 0.006257766731759275, 0.28368542517308715, 0.0, 1.339605893839872, 0.12777383277116988, 3.9999999999999996, 0.0, 0.04895260074560624, 0.3785726966092668, 1.073096041185869, 0.2827977986863128, 0.1283507899875732, 0.0, 0.27618498135984376, 0.0, 1.4842002485354162, 1.1893751109533108, 1.9754571276406887, 0.06763713829220663, 2.9116811645659504, 0.13034794958281554, 220.6216492100124, 0.01815196165453577, 0.44962719687555475, 0.7794248180365702, 0.10269838451979406, 11.53492810225457, 0.9999999999999999, 0.8545180188176814, 8.876264867743653e-05, 1.5016864903248712, 8.876264867743653e-05, 0.49840227232380613, 0.6753062311379371, 0.23810580507722348, 0.20588496360731404, 0.05121604828688088, 0.0, 0.03679211787679744, 0.1557340671045624, 0.45996804544647607, 0.0, 3.2920734954731046, 0.44012959346706904, 8.876264867743653e-05], [4.5339165226692995, 2.9033880523327573, 1.6540000411421047, 0.15721426808195507, 0.01998477742121287, 2.19059080062536, 2.3341458899037275, 1.9120998930305273, 1.9614189911955895, 0.09868448119805809, 0.04190323376943964, 1.3304945281000575, 0.0848555912120464, 2.745299514523163, 2.4429153295482595, 0.1269079651114951, 1.6130327902575494, 0.09325372336048712, 0.01153007487863079, 0.29429976137579195, 0.08440817082201925, 0.023759565539372994, 0.06833703612276804, 0.05513556323541512, 6.864760758660412, 0.039912984448284373, 1.9304081296799143, 0.37804451575742615, 1.134827820291286, 0.02970974245042376, 1.4797015140294576, 3.4823551797909977, 1.5619754381634163, 1.9742758989549904, 2.656247428618448, 0.001537686168024356, 1.9641395128774788, 0.01998477742121287, 1.8609139718588001, 1.1774458981321483, 1.5483214021229326, 2.605997490331605, 1.5335102443841027, 0.053762445486711094, 0.0, 0.7951637455772237, 2.032538262157492, 0.0, 2.8260408952522007, 22.52155846293096, 0.00344565127951946, 2.4886807784086233, 1.23639224882745, 1.148091006335884, 0.049745947502674234, 0.48086377849090756, 0.016796264296881427, 0.01073294659754793, 0.566125647988151, 2.5203910557064098, 0.07768657944540443, 1.407857113469925, 1.0972959351600426, 1.1821360980827778, 0.045832304780712575, 11.19922035711347, 1.3210472722784496, 0.28127828519707065], [1.4014231896190874, 3.7153620761825032, 1.2503139388865634, 0.05860192549183759, 3.030975303474257, 0.10087902888237757, 0.15403934700711594, 0.3482628714943491, 0.34910004185851823, 1.1326915027208038, 0.0, 0.13478442863122644, 1.156969443281708, 0.0849727919631645, 0.04981163666806195, 2.5893679363750524, 0.05525324403516116, 0.0016743407283382169, 0.0, 0.00795311845960653, 0.00795311845960653, 0.010464629552113856, 0.0012557555462536626, 0.011301799916282964, 0.4889074926747593, 0.0, 1.0958560066973628, 0.20636249476768523, 4.0, 0.0, 0.11176224361657597, 0.7011301799916283, 1.472164085391377, 0.5307660108832147, 0.22561741314357472, 0.0, 0.5207199665131854, 3.3131017161992466, 1.5504395144411889, 1.2687316868982839, 1.9782335705316032, 0.11050648807032232, 2.730431142737547, 0.336123901213897, 219.49518627040604, 0.044370029300962746, 0.8137295939723734, 0.8267057346169946, 0.16617831728756802, 46.407701967350356, 1.0, 1.6023440770196735, 0.0, 1.7743825868564254, 0.0, 0.49686061113436586, 0.5274173294265383, 0.19464210966931772, 0.316450397655923, 0.11929677689409796, 0.0, 0.06195060694851402, 0.2959397237337798, 0.6035998325659272, 0.0, 4.385935537881959, 0.8195897865215571, 0.0], [5.297841020608439, 3.6157180242067386, 1.1603696434412822, 0.19786555446516194, 3.474362119725221, 0.4543261367353615, 0.0007769054628720968, 1.7427625122669284, 1.7714262348707883, 1.729146221786065, 0.0022489368662087015, 2.48286719005561, 0.7332351979064442, 0.2625531566895649, 0.0005315668956493295, 6.158488714425908, 0.10929833169774289, 0.00670592083742231, 0.0011858030749100426, 0.16854759568204122, 0.3307572783774943, 0.11207883545960091, 0.11768073274452077, 0.0353696434412823, 1.42435394177298, 0.003925417075564279, 1.2491413150147204, 1.6980700032711809, 1.039213280994439, 0.0009404645076872752, 0.00032711808963035657, 3.836808963035656, 1.4802911350997712, 1.8406526005888126, 0.9261939810271508, 0.0028213935230618254, 1.846949623814197, 3.2409633627739614, 1.6667893359502781, 1.2265701668302258, 2.397448478900883, 0.2193326790971541, 2.426970886490023, 0.3307572783774943, 0.0, 0.0007360157016683023, 5.690873405299313, 0.0, 1.2475466143277725, 51.06501472031403, 0.0165603532875368, 2.41519463526333, 0.34457801766437685, 1.148102715080144, 0.0013493621197252208, 0.7066568531239777, 0.047636571802420674, 0.04105332024860975, 2.499836440955185, 0.0009404645076872752, 0.002657834478246647, 0.11952077199869153, 1.8526333006215245, 1.8568040562643116, 0.028581943081452406, 7.668588485443245, 5.216879293424927, 0.044242721622505726]])
#3
cluster_centers = np.array([[5.294358068557579, 3.6176343732670624, 1.1686007106327918, 0.20803466902175027, 3.4768222801864894, 0.45480498675265457, 0.0012323111996549528, 1.7431452689519193, 1.7706463472242189, 1.7252767565569225, 0.0023208527593501613, 2.4682987943888763, 0.7177185811990388, 0.2722996980837561, 0.0010474645197067099, 6.158475220275626, 0.11236624288853744, 0.006428556758200004, 0.0015609275195629401, 0.16868286471276878, 0.3304853252274641, 0.11179116432869847, 0.11474871120787035, 0.035449485510074145, 1.438764402637146, 0.003573702478999363, 1.2473453962907433, 1.6954753640452669, 1.036188872229867, 0.0010269259997124607, 0.0005340015198504796, 3.839224465485017, 1.4806013678654315, 1.8381153854053276, 0.9333114255786727, 0.002587853519275401, 1.846310254883033, 3.243381461931853, 1.671342603052024, 1.2250611020969828, 2.3882396434512927, 0.22543079545687936, 2.423052434841545, 0.32855470434800466, 0.0, 0.0009242333997412146, 5.685966029287929, 0.0, 1.2515557928895644, 51.09626404321305, 0.016656739715336114, 2.4109347080449384, 0.3418020497442954, 1.1503419663579042, 0.0013555423196204481, 0.7036702335229723, 0.04820390642650291, 0.04142619482840066, 2.495635564501222, 0.0014582349195916941, 0.00314239355912013, 0.12224527100577132, 1.8509930374417218, 1.858838752079525, 0.02840477315204666, 7.6697405984924725, 5.202797346423217, 0.044301587627595555], [1.211319380050338, 4.979939395946483, 1.5904258842230758, 0.047158564048218304, 0.12201119353556762, 0.07249304543648165, 0.11831037223473308, 0.2066995628560074, 0.20851271691614784, 0.15349715194065439, 0.0, 0.08141806861836005, 0.23312690422572527, 0.06990164260166909, 0.034383693204397936, 0.10459994701284939, 0.045924956947940125, 0.0009438336203470659, 4.139621141873096e-05, 0.007269174725129156, 0.0077576500198701815, 0.005646443237514903, 0.0011508146774407207, 0.008742879851635979, 0.3703056696251159, 0.0, 1.3241240561663796, 0.12468538879321764, 4.0, 0.0, 0.0729318452775202, 0.4193105047026096, 1.1171761160418598, 0.31269042257252616, 0.1719929792025434, 0.0, 0.3080706053781958, 0.13322128758775997, 1.5728656113392503, 1.144870181480991, 1.9506722744734402, 0.09276890978937608, 2.803500463637568, 0.11309444959597298, 220.11078454099882, 0.02856338587892436, 0.45901775069545636, 0.805975957080408, 0.1335110610676911, 13.23128063319645, 1.0, 0.9434858921711485, 0.00016558484567492384, 1.5579298582593721, 0.00016558484567492384, 0.490801761822758, 0.5915270234468142, 0.20849615843158034, 0.19965392767253942, 0.07154921181613459, 0.0, 0.05101669095244403, 0.1635150351039873, 0.44360180156312096, 0.0, 3.4227132732812295, 0.4328387865942509, 0.00016558484567492384], [4.467893684765287, 2.8413348343619123, 1.6621563044759549, 0.11116135329576604, 0.01349366480701284, 2.267481421652857, 2.81674561619614, 1.9509307104833955, 1.9976050070321734, 0.08184763728330459, 0.04555476207553583, 1.2048405053185682, 0.07135395064692876, 3.0844141466746082, 2.9533662436967716, 0.08437360642905925, 1.8370812270596784, 0.10484331195095253, 0.012433381461881254, 0.30564226663631344, 0.04276995905435199, 0.01264855661133443, 0.04569509369474442, 0.05229067979754825, 7.0280694423221455, 0.038347953809067914, 1.9394110437865837, 0.1024264896201379, 1.1192943502490107, 0.03423467812604274, 1.795702110275705, 3.486143032223258, 1.5678877596524765, 1.993494849829575, 2.6367749922817607, 0.001297287857572762, 1.9805219712538473, 0.01349366480701284, 1.9012346064009928, 1.183101578262944, 1.4559842080151184, 2.9290265039651477, 1.3789140827457596, 0.04610985159151648, 0.0, 0.9612123404507451, 1.3109811051270936, 0.0, 3.0244021093401607, 22.453021651609603, 0.0003804546120766273, 2.4929101347495393, 1.1556807798695852, 1.132996953244623, 0.05840913839504286, 0.45211105532496126, 0.015380345465261687, 0.009829450305455158, 0.15353838381633397, 3.046842694491828, 0.08737670308012312, 1.5873190111922262, 1.0246453508134556, 1.0369727039408236, 0.030442605926983897, 11.503575337809393, 1.0459726384527348, 0.29025568421019804], [1.2409933717185033, 3.2022019656854823, 1.7592719253520368, 0.03346473828858518, 0.19985988998462106, 0.08967455510924852, 0.16903983186798155, 0.2185799145246001, 0.21993541674439043, 0.14524185558719777, 0.0, 0.08209700753194965, 0.14068206218729146, 0.08421523882954249, 0.04559378873233599, 0.16768432964819122, 0.05857676412188743, 0.0013306306194272072, 0.0002528612703584412, 0.019064081678335593, 0.0018736605606887776, 0.0011026409494318912, 0.0002570065370856288, 0.01154042256849017, 0.4462048010479234, 0.0, 1.394853236831524, 0.12113298430187491, 4.0, 0.0, 0.0867811589336716, 0.4407869374354893, 1.0680611343936925, 0.33402973814350084, 0.21689693623336195, 0.0, 0.33193223317954396, 0.20970075319496434, 1.9471271228947227, 1.0792865166909165, 1.9419455394857383, 0.118902830802648, 2.0965722789432886, 0.007204473571851981, 111.83873668851223, 0.04283304109202907, 0.4429507666670812, 0.9925633914914256, 0.15920311392436548, 14.199901342651893, 1.0, 1.005061370673896, 9.119586799812635e-05, 1.6844125535257568, 9.119586799812635e-05, 0.4864885031027322, 0.025493390372203502, 0.009281252202172949, 0.18974543917028341, 0.09046630105414133, 0.0, 0.06684242597589943, 0.16070784574633457, 0.4114591753406373, 0.0, 3.595264447290861, 0.39455063236043925, 9.119586799812635e-05], [4.315284004705, 1.0752395117710234, 1.6765291249722984, 0.14670522152707932, 0.0, 2.1034460714955423, 2.6625718109135543, 1.949108010432826, 1.993534886892484, 0.08331344504866948, 0.030062562861185457, 1.1891141473892366, 0.06180000340942023, 2.9235075262951535, 2.695741207957587, 0.0, 1.6681376894359115, 0.09455813913843951, 0.027545984555326367, 0.2870902303063364, 0.04833492439610645, 0.014686077632498594, 0.03616542506946694, 0.052607354119431994, 6.562471233016826, 0.030292698726581545, 1.9355917901160908, 0.1843580061710506, 1.2622312950682737, 0.011999028315234994, 1.5888899780092396, 3.664466170027787, 1.4285939550979356, 1.9919942551269159, 2.9873190876391473, 0.0010505276077802971, 1.979306950103135, 0.0, 1.923045123676719, 1.0943748828011797, 1.4598647312524506, 2.759205434615844, 1.2634437701368884, 0.04365975690833774, 0.0, 0.9289540751095277, 1.437569253848383, 0.0, 2.852195240449362, 10.0, 0.0007287635737542831, 2.657630282470466, 0.9017831267792912, 1.1431146758493718, 0.01582610252126626, 0.457369461823017, 0.02283033020234909, 0.014411193126608822, 0.2748333645863521, 2.7884795690492834, 0.056570805134586864, 1.5623582959717701, 1.029700311961951, 1.0773448287618692, 0.027006869981759603, 10.86571359165374, 1.0920927021360018, 0.1716962717989806], [4.092577020105496, 10.905331118694011, 1.171483970505452, 0.28223446914678474, 0.0, 1.909063110270969, 2.1418087302194024, 1.8477180643681677, 1.8952818452856677, 0.06809320944325914, 0.017582753612109165, 1.3885719051226275, 0.02085606466005518, 2.4660194174757284, 2.108123510156994, 0.0, 1.300428796797576, 0.05582698014469286, 0.013558874425780626, 0.13137027333189708, 0.0631519691989075, 0.045847203786199275, 0.03791064069330257, 0.05308184667560862, 6.237353274353504, 0.018646058474817744, 1.9289948641680161, 0.4036249661201882, 1.6388655301582449, 0.01410095141461245, 1.229097024831296, 3.5903496396577967, 1.440395855195947, 1.9135178711663692, 3.30163804546497, 0.000827014893217783, 1.8863375749699427, 0.0, 1.7861089296759354, 1.1520873438922519, 1.5630372990666548, 2.2150447213515787, 1.7473573746794449, 0.12241905331118695, 0.0, 0.8007102598494694, 1.9707000437831415, 0.0, 2.3764029716938517, 10.000076446754836, 0.024372615382477016, 2.9477451682176095, 0.6438484686325066, 1.1501900744313405, 0.019653765697646135, 0.5026582621567715, 0.054242447408107526, 0.03135706889242552, 0.6374477903413001, 2.383915602782662, 0.043317511171650766, 1.3095537594429116, 1.0768011897894936, 1.198198636467882, 0.020313987671223357, 9.647941844868686, 1.2953346630435538, 0.13868136297614167], [5.62021151872391, 2.7889721423268683, 1.4449114209563374, 0.09943489531567568, 0.0, 0.4425585084157191, 0.00010547538975184887, 1.6045889908033575, 1.644515482570192, 0.12676113475292392, 0.008003959383861455, 2.2864873855490604, 0.0658166432051537, 0.18505251457385916, 0.00013387261006965432, 0.0, 0.08055480055009473, 0.006044551181932877, 0.005947189283700401, 0.24245141032766337, 0.506780850537316, 0.10460318941351628, 0.1974418161239417, 0.05540703358579815, 1.2929416680527215, 0.019902394697021944, 1.8935996722149426, 1.7387739702965077, 1.1075159328689712, 0.0019431812188898312, 0.00026774522013930864, 3.5929866979306544, 1.267031232885604, 1.7858687318207083, 0.7693090956296679, 0.0034888013533303855, 1.8189474367451919, 0.0, 1.6783244017314192, 1.0419954321042746, 2.3984292280418495, 0.17166525356689372, 1.8225011460306773, 0.11756449211571462, 0.0, 5.679444063561093e-05, 5.7445710599871, 0.0, 1.5869178062741631, 10.0, 0.018746222155511295, 2.413573059962759, 1.253972568285173, 1.0988060997229243, 0.002101394303517604, 0.6695090931956204, 0.026068648251745416, 0.016348685411536574, 2.5552224516537327, 8.924840671310289e-05, 0.008438031180147909, 0.09192585891449598, 1.8532959030924574, 1.8618637501369153, 0.10612041232763902, 8.360535165900618, 5.0835933031241005, 0.1544321975797455], [4.295768115942029, 4.125554677206852, 1.2062924901185772, 0.15978919631093544, 3.46331488801054, 1.9969960474308301, 2.7358840579710146, 1.9709407114624506, 1.9961739130434784, 1.4491172595520423, 0.008368906455862978, 1.1274729907773386, 0.8225876152832675, 2.8800632411067197, 2.7177338603425563, 5.173575757575758, 1.574144927536232, 0.08602898550724639, 0.005934123847167325, 0.20034782608695653, 0.03150461133069829, 0.013270092226613966, 0.020627140974967064, 0.03124110671936759, 6.51970487483531, 0.0056916996047430835, 1.2449749670619237, 0.16768379446640316, 1.2540395256916996, 0.009306982872200264, 1.8887905138339922, 3.877955204216074, 1.607209486166008, 1.9885428194993413, 3.142872200263505, 0.00042160737812911727, 1.9524321475625825, 3.4049749670619236, 1.8654018445322793, 1.2734440052700924, 1.3227931488801055, 2.625422924901186, 1.904874835309618, 0.21556785243741766, 0.0, 0.9538761528326746, 1.404121212121212, 0.0, 2.6988774703557312, 50.712274044795784, 0.0013807641633728591, 2.6807061923583664, 0.24665085638998685, 1.1903346508563901, 0.0169802371541502, 0.4209749670619236, 0.036816864295125165, 0.02865876152832675, 0.24862187088274046, 3.180332015810277, 0.018772068511198947, 1.4554519104084322, 1.055820816864295, 1.0516785243741766, 0.006492753623188406, 10.03372859025033, 1.0966640316205534, 0.06798418972332017], [5.714427265738447, 3.0784374021240852, 1.5469662025568747, 0.16964778907206512, 0.020107627914922696, 0.7157711910253125, 0.0007744654195495572, 1.658996042253922, 1.7083682127502064, 0.1173884570484895, 0.018427721306340936, 2.2251758207340337, 0.08231542382050624, 0.43713447794766663, 0.0005011246832379488, 0.12813985934341277, 0.20544973093021268, 0.015284302838757438, 0.0031491130662566554, 0.2857435722217477, 0.49827738390136955, 0.08495202300617864, 0.2230574300276188, 0.06195153896529142, 2.3806953104979924, 0.03584180404885966, 1.918988639275647, 1.7528942797756328, 0.9317730132968879, 0.005142222601862133, 0.0008143276102616668, 3.564545428660915, 1.3978929984909312, 1.8354545713390849, 1.1173258164630848, 0.005432647134193218, 1.866900145212266, 0.020107627914922696, 1.7223370632954642, 1.126682042083084, 2.3399162893995045, 0.4071581105321602, 1.8362973719427123, 0.10344238489792432, 0.0, 0.00041570570314057115, 5.766350616440306, 0.0, 1.8326072720025055, 22.505270351072006, 0.00989151789527633, 2.2771845904159904, 1.5896927764015831, 1.111591355599214, 0.004868881865550525, 0.6575666979869593, 0.016502946954813358, 0.011292389168873323, 2.573058853677287, 0.000615016656701119, 0.019355940890065773, 0.21237436291677342, 1.7767090914267816, 1.894450613593007, 0.13283220865009537, 9.305065345519774, 4.568765126277725, 0.21816576976737564], [1.1409101742779346, 3.2056234065449924, 1.7796572220978355, 0.02244017998367724, 0.04415089161255841, 0.05335230432553067, 0.08917570075908043, 0.15644568349740354, 0.15771683627407787, 0.13674281545895114, 0.0, 0.05909415919744036, 0.13254656680413412, 0.04987107910759297, 0.026029742085990595, 0.03536115906021364, 0.03333164809290971, 0.000693356060004189, 0.00013000426125078543, 0.011917057281321999, 0.004709043240861784, 0.002340076702514138, 0.00045501491437774903, 0.006139090114620423, 0.25587727597737925, 0.0, 1.2130408718952455, 0.1006305206670663, 4.0, 0.0, 0.04815213387549925, 0.3170731707317073, 0.9413897455527709, 0.24152625002708422, 0.12884144535848674, 0.0, 0.23963396577998944, 0.04320474948901103, 1.8358840650888002, 1.0785009064185993, 1.933134474963346, 0.06761666076832518, 2.0941808648172358, 0.00941086402276519, 123.14104740099815, 0.02277963555472096, 0.355807218125483, 0.9888918581220162, 0.10007439132727128, 12.5889987505146, 1.0, 0.7283272062806503, 7.222458958376968e-05, 1.5196125873014725, 7.222458958376968e-05, 0.484634218566053, 0.03773012559856129, 0.013563777923831948, 0.15568732530677395, 0.050297204186137215, 0.0, 0.03765067855001914, 0.12469575391637837, 0.32816686769177433, 0.0, 3.0251486020930685, 0.3382783102335021, 7.222458958376968e-05]])

In [22]:
import os

output_dir = "experiment_results_US_Census"
os.makedirs(output_dir, exist_ok=True)

X_full = df

class KMeansWrapper:
    def __init__(self, cluster_centers_ , n_clusters=10, random_state=43):
        self.n_clusters = n_clusters
        self.cluster_centers_ = cluster_centers_
        self.random_state = random_state
        self.labels_ = None

    def fit(self, X):
        # Use the 'prediction' column as cluster labels
        if 'prediction' not in X_full.columns:
            raise ValueError("The dataframe must contain a 'prediction' column.")
        self.labels_ = X_full.loc[X.index, 'prediction'].values
        return self

    def predict(self, X):
        # Return the stored labels for the given indices
        return self.labels_

def measure_runtime_and_save_tree(feature_count, data_points):
    # Select first `feature_count` columns and `data_points` rows
    X_subset = X_full.iloc[:data_points, :feature_count]
    print(X_subset.columns)
    # X_subset.to_csv(os.path.join(output_dir, f"subset_features_{feature_count}_datapoints_{data_points}.csv"), index=False)

    # Preprocess the dataset: only standardize numerical features
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), X_subset.columns)
        ]
    )

    X_preprocessed = preprocessor.fit_transform(X_subset)
    X_preprocessed = pd.DataFrame(X_preprocessed)

    # Fit KMeansWrapper model
    k = 10  # Number of clusters
    kmeans = KMeansWrapper(cluster_centers_ =cluster_centers , n_clusters=k, random_state=43 )
    kmeans.fit(X_subset)

    # Measure the runtime for Tree creation
    start_time = time.time()
    tree = Tree(k=k)
    tree.fit(X_preprocessed, kmeans)
    end_time = time.time()

    tree_filename = f"tree_features_{feature_count}_datapoints_{data_points}"
    tree_filepath = os.path.join(output_dir, tree_filename)
    tree.plot(filename=tree_filepath, feature_names=list(X_subset.columns), view=False)

    score = tree.score(X_preprocessed)
    print(f"Tree Kmeans Cost score: {score}")

    surrogate_score = tree.surrogate_score(X_preprocessed)
    print(f"Tree Surrogate Cost score: {surrogate_score}")


    # Move the generated .gv and .gv.png files to the output directory if not already there
    for ext in [".gv", ".gv.png"]:
        src = f"{tree_filename}{ext}"
        dst = os.path.join(output_dir, f"{tree_filename}{ext}")
        if os.path.exists(src):
            os.replace(src, dst)

    Image(filename=f'{tree_filepath}.gv.png')

    return end_time - start_time, tree_filename

feature_counts = [68]
data_point_sizes = [2000000]

results = []

for feature_count in feature_counts:
    for data_points in data_point_sizes:
        print(f"Testing with {feature_count} features and {data_points} data points...")
        runtime, tree_filename = measure_runtime_and_save_tree(feature_count, data_points)
        print(f"Runtime: {runtime} seconds, Tree saved as {tree_filename}.gv.png")
        results.append({
            'Features': feature_count,
            'Data Points': data_points,
            'Runtime (seconds)': runtime,
            'Tree Filename': tree_filename
        })

results_df = pd.DataFrame(results)
print(results_df)

results_csv_path = os.path.join(output_dir, f'{data_point_sizes[0]}_runtime_and_tree_results.csv')
results_df.to_csv(results_csv_path, index=False)


Testing with 68 features and 2000000 data points...
Index(['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10',
       'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20',
       'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30',
       'f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40',
       'f41', 'f42', 'f43', 'f44', 'f45', 'f46', 'f47', 'f48', 'f49', 'f50',
       'f51', 'f52', 'f53', 'f54', 'f55', 'f56', 'f57', 'f58', 'f59', 'f60',
       'f61', 'f62', 'f63', 'f64', 'f65', 'f66', 'f67'],
      dtype='object')
Tree Kmeans Cost score: 85033012.34718595
Tree Surrogate Cost score: 8189059345.276954
Runtime: 63.785183906555176 seconds, Tree saved as tree_features_68_datapoints_2000000.gv.png
   Features  Data Points  Runtime (seconds)  \
0        68      2000000          63.785184   

                         Tree Filename  
0  tree_features_68_datapoints_2000000  
